<H2>Create a single span bridge deck - v21.1 and later</H2>

In [53]:
import numpy as np
import math
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0") # Connect to LUSAS Modeller

# If there is not currently a model open, create one
if not lusas.existsDatabase():
    lusas.newProject("Structural", "Bridge Deck")

db = lusas.database() # Reference to the model database for convenience

# Delete all previous geometry and attributes/utilites
db.deleteAll()
db.deleteAllAttributes()

# 3D model with Z vertical
db.setAnalysisCategory("3D")

# Set the unit system
db.setModelUnits("kN,m,t,s,C")

Define model parameters

In [54]:
deck_width = 18
deck_length = 25
no_beams = 4
deck_overhang = 1.5
skew_angle = 10
deck_thk = 0.2

mesh_size = 1


Create the attributes to be assigned to the geometric features. Mesh, Geometric and Material.

In [55]:
# Deck surface attributes
deck_mesh_attr    = db.createMeshSurface("SMsh1").setRegularSize("QTS4", mesh_size, True)
deck_surface_attr = db.createGeometricSurface("Deck Slab").setSurface(deck_thk, 0.0)
deck_mat_attr     = db.createIsotropicMaterial("Concrete deck", 34.8E6, 0.2, 2.4)

# Beam mesh attributes
beam_mesh_attr = db.createMeshLine("LMsh2").setSize("BMI21", mesh_size)
beam_geom_attr = db.createGeometricLine("LGeo2").setFromLibrary("Australian Sections", "Precast Super T (Open)", "T3 (100)", 0, 0, 0)
beam_geom_attr.setValue("elementType", "3D Thick Beam")
beam_geom_attr.setEccentricityOrigin("Centroid", "Fibre", "", "A1")
beam_mat_attr = db.createIsotropicMaterial("Concrete beam", 34.8E6, 0.2, 2.4)

# Support attributes
support_pinned_attr = db.createSupportStructural("Pinned").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F") # F=Free, R=Restrained
support_slide_attr  = db.createSupportStructural("Slide").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")


Define some useful helper functions for creating lines. This keeps the code clean and avoid duplication

In [56]:
# Define a useful helper function to create a line from two point coordinates
def create_line(p1:list, p2:list) -> 'IFLine':
      
        geomData = lusas.geometryData()            # geometryData object contains all the settings to perform a geometry creation
        geomData.setAllDefaults()                  # Reset the inputs
        geomData.addCoords(p1[0], p1[1], p1[2])    # Set the coordinates of the first point
        geomData.addCoords(p2[0], p2[1], p2[2])    # Set the coordinates of the second point
        geomData.setLowerOrderGeometryType("coordinates")
        geomData.setCreateMethod("straight")
        # Create the line and add it to the group
        return win32.CastTo(db.createLine(geomData), "IFObjectSet").getObjects("Line")[0]


# Define a useful helper function to create a line from two point objects
def create_line_from_points(p1:'IFPoint', p2:'IFPoint') -> 'IFLine':
      
        geomData = lusas.geometryData()            # geometryData object contains all the settings to perform a geometry creation
        geomData.setAllDefaults()                  # Reset the inputs
        geomData.setLowerOrderGeometryType("points")
        geomData.setCreateMethod("straight")

        obs = lusas.newObjectSet()                 # Create an object set to contain the points and use this set to create the line
        obs.add(p1)
        obs.add(p2)

        # Create the line and return it
        return win32.CastTo(obs.createLine(geomData), "IFObjectSet").getObjects("Line")[0]

Helper function to calculate beam coordinates accounting for the skew angle

In [57]:
def get_x(y:float)->float:
    return y * math.tan(math.radians(skew_angle))

Calculate the x and y coordinates of the frame grid

In [58]:
# Calculate the beam spacing 
beam_spacing = (deck_width - 2*deck_overhang) / (no_beams-1)
print(f"Beam Spacing = {beam_spacing}")
assert beam_spacing > 0, "Cant have a negative beam spacing"

# Y cordinates of beams between the overhangs
beam_ys = [deck_overhang]
beam_ys.extend([float(y) for y in np.add.accumulate([beam_spacing]*(no_beams-1)) + deck_overhang])
print(f"Beam y coordinates = {beam_ys}")


# Create the longitudinal lines storing them in a list for reference
longitudinal_lines = []

# First the near edge line
longitudinal_lines.append( create_line([0, 0, 0], [deck_length, 0, 0]) )

# Now the internal beam lines, here we'll also assign the beam attributes
for y in beam_ys:
    line = create_line([get_x(y), y, 0], [get_x(y) + deck_length, y, 0])
    beam_mesh_attr.assignTo(line)
    beam_geom_attr.assignTo(line)
    beam_mat_attr.assignTo(line)
    longitudinal_lines.append(line)

# And the far side edge line
longitudinal_lines.append( create_line([get_x(deck_width), deck_width, 0], [get_x(deck_width) + deck_length, deck_width, 0]) )


# Now the start end support lines
end1_lines = []
for i in range(0, len(longitudinal_lines)-1):
    end1_lines.append(create_line_from_points(longitudinal_lines[i].getStartPoint(), longitudinal_lines[i+1].getStartPoint()))

# And opposite end support lines
end2_lines = []
for i in range(0, len(longitudinal_lines)-1):
    end2_lines.append(create_line_from_points(longitudinal_lines[i].getEndPoint(), longitudinal_lines[i+1].getEndPoint()))


Beam Spacing = 5.0
Beam y coordinates = [1.5, 6.5, 11.5, 16.5]


Define helper function for creating surfaces.

In [59]:
def create_surface_from_lines(lines: list):

    geometryData = lusas.geometryData()
    geometryData.setAllDefaults()
    geometryData.setCreateMethod("coons")
    geometryData.setLowerOrderGeometryType("lines")

    obs = lusas.newObjectSet()
    for p in lines:
        obs.add(p)

    objs = obs.createSurface(geometryData).getObjects("Surface")

    # Assign the mesh attribute to the created surfaces
    deck_mesh_attr.assignTo(objs)
    # Assign the geometric thickness attribute to the created surfaces
    deck_surface_attr.assignTo(objs)
    # Assign the material attribute to the created surfaces
    deck_mat_attr.assignTo(objs)

Create surfaces between the deck lines and assign the mesh and thickness attributes

In [60]:
for i in range(0, len(longitudinal_lines)-1):
    # Create a list of lines in clock-wise order from which to create the surface
    lines=[]
    lines.append(longitudinal_lines[i])
    lines.append(end1_lines[i])
    lines.append(end2_lines[i])
    lines.append(longitudinal_lines[i+1])
    create_surface_from_lines(lines)

Generate the mesh from the assignments

In [61]:
db.resetMesh()
db.updateMesh()

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFDatabase instance at 0x1654334231600>

In [62]:
for i in range(1, len(longitudinal_lines)-1):
    support_pinned_attr.assignTo(longitudinal_lines[i].getStartPoint())
    support_slide_attr.assignTo(longitudinal_lines[i].getEndPoint())

Assign gravity and solve

In [63]:
# getLoadset returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity funtion
win32.CastTo(db.getLoadset("Loadcase 1", 0), "IFLoadcase").addGravity(True) 

db.getAnalysis("Analysis 1").solve(True)
db.openAllResults(False)



Create beam/shell slicing to compute equivalent beam results

In [64]:
slice = db.createBeamShellSlice("Girder 1")
slice.setBeamShellSlice(4, "1", -1, 0.0, -1, -1, beam_spacing, 0, -1, "Slice", 0, "", "2", 0.0)
slice = db.createBeamShellSlice("Girder 2")
slice.setBeamShellSlice(4, "1", -1, 0.0, -1, -1, beam_spacing, 0, -1, "Slice", 0, "", "3", 0.0)

Plot diagrams of the beam shell resultants

In [65]:
lusas.view().insertDiagramsLayer()
diagrams_layer = lusas.view().diagrams()
diagrams_layer.setResultsTransformNone()
diagrams_layer.setResults("Beam/Shell Slice Resultants", "My")
diagrams_layer.setLocation("BeamShellSlice")
diagrams_layer.setAssocOption("Single")
